### a toy CNN autoencoder with Keras.
1. Learning TensorFlow: A Guide to Building Deep Learning Systems (Kindle Locations 4623-4624). O'Reilly Media. Kindle Edition. 
2. current practical applications are mostly for extracting lower-dimensional representations, denoising data, and data visualization with reduced dimensionality. Denoising works because the network learns the important abstractions of the image, while losing unimportant image-specific signals like noise.


## Read automobile images and noise

In [ ]:
#import keras # keras now available from tensorflow
from tensorflow import keras
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, LSTM
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.datasets import cifar10
import numpy as np

CLASS_AUTOMOBILE = 1
HORIZ_PIXELS = VERT_PIXELS = 32; NUM_OF_CHANNELS = 3

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

#x_train = x_train[np.where(y_train==CLASS_AUTOMOBILE)[0],:,:,:]
#x_test = x_test[np.where(y_test==CLASS_AUTOMOBILE)[0],:,:,:]

#The following works just as well as the ones above.

x_train = x_train[np.where(y_train==CLASS_AUTOMOBILE)[0]]
x_test = x_test[np.where(y_test==CLASS_AUTOMOBILE)[0]]

#print("x_train no of pictures: ", len(x_train), "Shape: ", x_train.shape)
#print("x_test no of pictures: ", len(x_test), "Shape: ", x_test.shape)
assert((HORIZ_PIXELS, VERT_PIXELS, NUM_OF_CHANNELS) == (x_train.shape[1], \
                                                        x_train.shape[2], \
                                                        x_train.shape[3]))
assert((HORIZ_PIXELS, VERT_PIXELS, NUM_OF_CHANNELS) == (x_test.shape[1], \
                                                        x_test.shape[2], \
                                                        x_test.shape[3]))

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

assert not x_train.any() > 1.0
assert not x_test.any() > 1.0
assert not x_train.any() < 0.0
assert not x_test.any() < 0.0

NOISE_MEAN = 0.0; NOISE_STD_DEV = 0.4
NOISE_GAIN = 0.5

x_train_plusNoise = x_train + NOISE_GAIN *\
                         np.random.normal(loc=NOISE_MEAN, \
                                          scale=NOISE_STD_DEV, \
                                          size=x_train.shape) 

x_test_plusNoise = x_test + NOISE_GAIN *\
                         np.random.normal(loc=NOISE_MEAN, \
                                          scale=NOISE_STD_DEV, \
                                          size=x_test.shape) 



In [ ]:
#print((x_test_plusNoise).max())
#print((x_train_plusNoise).max())
assert not(x_train_plusNoise.any() > ((x_train_plusNoise).max() - 0.0))
assert not(x_test_plusNoise.any() > ((x_test_plusNoise).max() - 0.0E-00))

In [ ]:
TOP_CLIP = 1.0; BOTTOM_CLIP = 0.0 #Pixel values are non-negative. They are unsigned byte.
# Keep the values in the range [0, TOP_CLIP]. 
# Why?
x_train_plusNoise = np.clip(x_train_plusNoise, 0., TOP_CLIP)
x_test_plusNoise = np.clip(x_test_plusNoise, 0., TOP_CLIP)

#https://www.tensorflow.org/api_docs/python/tf/keras/layers/InputLayer
#It is generally recommend to use the functional layer API via Input, 
#(which creates an InputLayer) without directly using InputLayer
inp_img = Input(shape=(HORIZ_PIXELS, VERT_PIXELS, NUM_OF_CHANNELS))   


In [ ]:
#This is a small SANDBOX.
#Taken from https://keras.io/getting-started/functional-api-guide/#the-concept-of-layer-node
a = Input(shape=(280, 256))

#lstm = LSTM(32)
lstm = Conv2D(32, #filters: Integer, the dimensionality of the output space
            (3, 3), # height and width of the 2D convolution window
            activation='relu', 
            padding='same')
encoded_a = lstm(inp_img)

assert lstm.output == encoded_a, "This should NOT fail"
# END of SANDBOX

## Define the layers of the model

In [ ]:
STD_NUM_OF_FEATURES = 32
NUM_OF_FEATURES = STD_NUM_OF_FEATURES * 2
CONV_WINDOW_EDGE = 3

conv2d_op= Conv2D(NUM_OF_FEATURES, #filters: Integer, the dimensionality of the output space
            (CONV_WINDOW_EDGE, CONV_WINDOW_EDGE), # height and width of the 2D convolution window
            activation='relu', 
            padding='same')
img = conv2d_op(inp_img)
assert conv2d_op.input == inp_img
assert conv2d_op.input_shape == (None, HORIZ_PIXELS, VERT_PIXELS, NUM_OF_CHANNELS)
assert conv2d_op.output == img
assert conv2d_op.output_shape == (None, HORIZ_PIXELS, VERT_PIXELS, NUM_OF_FEATURES)


In [ ]:
POOL_FACTOR = 4
max_pool_op = MaxPooling2D((POOL_FACTOR, POOL_FACTOR), padding='same')
img = max_pool_op(img)

assert max_pool_op.input_shape == (None,HORIZ_PIXELS,VERT_PIXELS,NUM_OF_FEATURES)
assert max_pool_op.output_shape == (None, HORIZ_PIXELS//POOL_FACTOR, VERT_PIXELS//POOL_FACTOR, NUM_OF_FEATURES)

In [ ]:
conv2d_op = Conv2D(NUM_OF_FEATURES, (CONV_WINDOW_EDGE, CONV_WINDOW_EDGE), activation='relu', padding='same')
img = conv2d_op(img)
assert conv2d_op.input_shape == (None, HORIZ_PIXELS//POOL_FACTOR, VERT_PIXELS//POOL_FACTOR, NUM_OF_FEATURES)
assert conv2d_op.output_shape == (None, HORIZ_PIXELS//POOL_FACTOR, VERT_PIXELS//POOL_FACTOR, NUM_OF_FEATURES)


In [ ]:
conv2d_op = Conv2D(NUM_OF_FEATURES, (CONV_WINDOW_EDGE, CONV_WINDOW_EDGE), activation='relu', padding='same')
img = conv2d_op(img)
assert conv2d_op.input_shape == (None, HORIZ_PIXELS//POOL_FACTOR, VERT_PIXELS//POOL_FACTOR, NUM_OF_FEATURES)
assert conv2d_op.output_shape == (None, HORIZ_PIXELS//POOL_FACTOR, VERT_PIXELS//POOL_FACTOR, NUM_OF_FEATURES)


In [ ]:
up_sampling_op = UpSampling2D((POOL_FACTOR, POOL_FACTOR))
img = up_sampling_op(img)
assert up_sampling_op.input_shape == (None, HORIZ_PIXELS//POOL_FACTOR, VERT_PIXELS//POOL_FACTOR, NUM_OF_FEATURES)
assert up_sampling_op.output_shape == (None, HORIZ_PIXELS, VERT_PIXELS, NUM_OF_FEATURES)

In [ ]:
decode_op = Conv2D(NUM_OF_CHANNELS, (CONV_WINDOW_EDGE, CONV_WINDOW_EDGE), activation='sigmoid', padding='same')
decoded = decode_op(img)

assert(decode_op.input_shape == (None, HORIZ_PIXELS, VERT_PIXELS, NUM_OF_FEATURES))
assert decode_op.output_shape == (None, HORIZ_PIXELS, VERT_PIXELS, NUM_OF_CHANNELS)

## Construct the Autoencoder.
1. The two ends of the sequence of definitions above are used to completely specify the layers of the model

In [ ]:
#https://www.tensorflow.org/api_docs/python/tf/keras/models
autoencoder = Model(inp_img, decoded)

#https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adadelta
#https://www.tensorflow.org/api_docs/python/tf/keras/losses/binary_crossentropy
#https://www.tensorflow.org/api_docs/python/tf/keras/backend/binary_crossentropy
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

## Construct the two callback functions to be used by the autoencoder

In [ ]:
tensorboard = TensorBoard(log_dir='./models/autoencoder',\
              histogram_freq=0, write_graph=True, write_images=True)

#Learning TensorFlow: A Guide to Building Deep Learning Systems (Kindle Location 4685). 
#O'Reilly Media. Kindle Edition. 
model_saver = ModelCheckpoint(
                    filepath='./models/autoencoder/epochs_50-NOISE_GAIN_0.5-MEAN_0.0-VAR_0.4',\
                     verbose=0, period=2)


## Train the autoencoder

In [ ]:
NO_OF_EPOCHS = 50
BATCH_SIZE = 64
autoencoder.fit(x_train_plusNoise, x_train,
                epochs=NO_OF_EPOCHS,
                batch_size=BATCH_SIZE,
                shuffle=True,
                validation_data=(x_test_plusNoise, x_test),
                callbacks=[tensorboard, model_saver])

## Test and Display

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

n_imgs = 10
_,axarr = plt.subplots(3,n_imgs,figsize=[20,5])
decoded_imgs = autoencoder.predict(x_test_plusNoise)

for i in range(n_imgs):
    
    ax = axarr[0,i]
    ax.get_yaxis().set_visible(False)
    ax.imshow(x_test_plusNoise[i,:,:,:])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    ax = axarr[1,i]
    ax.get_yaxis().set_visible(False)
    ax.imshow(decoded_imgs[i,:,:,:])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
            
    ax = axarr[2,i]
    ax.get_yaxis().set_visible(False)
    ax.imshow(x_test[i,:,:,:])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
            
plt.tight_layout()
plt.show()